<div>
<img src="./images/sunpy_logo.png" width="500" align="left"/>
</div>

# 1. Searching and downloading data with sunpy



In this notebook, an introduction to how you can search for and download data with sunpy. We will begin with an intoduction to `astropy.units` (which are used throughout the sunpy ecosystem), and then look about how to use `Fido` and build queries for data. In particular, this notebook will look at the following:

1. Introduction to `astropy.units`
2. Overview of `Fido` 
3. Constructing a data search query and inspecting it
4. More complex queries and the HEK
5. Extending Fido - the SOAR archive

In [1]:
import astropy.units as u
from sunpy.net import Fido, attrs as a
from sunpy.time import parse_time

import numpy as np

# 1.1 Astropy Units
[`astropy.units`](https://docs.astropy.org/en/stable/units/) provides a means to deal with and handle numbers/arrays etc that have an associated physical quantity (e.g. km, seconds, Kelvin). Throughout SunPy, any physical input or outputs is an [`astropy.Quantity`](https://docs.astropy.org/en/stable/units/quantity.html#quantity). Lets look at how we can create and convert between astropy units. Above we have imported `astropy.units` as `u`

In [2]:
distance_in_km = 10*u.km

In [3]:
distance_in_km

<Quantity 10. km>

In [4]:
distance_in_km.unit

Unit("km")

In [5]:
distance_in_km.value

10.0

We can convert between equivalent units

In [6]:
distance_in_km.cgs

<Quantity 1000000. cm>

In [7]:
distance_in_km.to(u.parsec)

<Quantity 3.24077929e-13 pc>

In [8]:
distance_in_km.to(u.Mm)

<Quantity 0.01 Mm>

However you can only convert between physical units that make sense for example:

In [9]:
distance_in_km.to(u.second)

UnitConversionError: 'km' (length) and 's' (time) are not convertible

In [ ]:
time_in_sec = 60*u.s

In [ ]:
(distance_in_km/time_in_sec).unit

## 1. 2 Overview of `Fido` Unified Downloader


* [`Fido`](https://docs.sunpy.org/en/stable/guide/acquiring_data/fido.html#fido-guide) is sunpy's interface for searching and downloading solar physics data.

* It offers a unified interface for searching and fetching data irrespective of the underlying client or webservice from where the data is obtained.

* Offers a way to search and accesses multiple instruments and all available data providers in a single query.

* It supplies a single, easy, consistent and *extendable* way to get most forms of solar physics data the community need 

Fido offers access to data available through:

 * **VSO**
 * **JSOC** (through `drms`)
 * **Individual data providers** from web accessible sources (http, ftp, etc)
 * **HEK**
 * **HELIO**
 
As described here `Fido` provides access to many sources of data through different clients, these clients can be defined inside sunpy or in other packages. Lets print the current list of available clients within sunpy.

In [10]:
Fido

Client,Description
CDAWEBClient,Provides access to query and download from the Coordinated Data Analysis Web (CDAWeb).
EVEClient,Provides access to Level 0CS Extreme ultraviolet Variability Experiment (EVE) data.
GBMClient,Provides access to data from the Gamma-Ray Burst Monitor (GBM) instrument on board the Fermi satellite.
XRSClient,Provides access to the GOES XRS fits files archive.
SUVIClient,Provides access to data from the GOES Solar Ultraviolet Imager (SUVI).
...,...
NoRHClient,Provides access to the Nobeyama RadioHeliograph (NoRH) averaged correlation time series data.
RHESSIClient,Provides access to the RHESSI observing summary time series data.
HEKClient,Provides access to the Heliophysics Event Knowledgebase (HEK).
HECClient,Provides access to the HELIO webservices.


### Using attributes to search for data with Fido

Sunpy uses specified **attributes** to search for data using Fido. The range of these attributes is located in the `attrs` submodule. These `attr` parameters can be combined together to construct data search queries, such as searching over a certain time period, for data from a certain instrument with a certain wavelength etc.

Different clients and provides will have client-specific attributes, but the core attributes are:

* `a.Time`
* `a.Instrument`
* `a.Wavelength`


Lets look at how these attributes work in more detail.

First we can look at `a.Time`, which is used to specify the timerange of a query.

In [11]:
a.Time("2022-03-28 11:00", "2022-03-28 14:00")

<sunpy.net.attrs.Time(2022-03-28 11:00:00.000, 2022-03-28 14:00:00.000)>

We can inspect the instrument attribute to see what instrument `attrs` are currently supported through sunpy. Here we can see the instrument name (i.e. the name to be passed to the `a.Instrument` attribute, the client from which the data is available to access, and the full name of the instrument.)

In [12]:
a.Instrument

Attribute Name,Client,Full Name,Description
aia,VSO,AIA,Atmospheric Imaging Assembly
bcs,VSO,BCS,Bragg Crystal Spectrometer
be_continuum,VSO,BE-Continuum,INAF-OACT Barra Equatoriale Continuum Instrument
be_halpha,VSO,BE-Halpha,INAF-OACT Barra Equatoriale Hα Instrument
bigbear,VSO,Big Bear,"Big Bear Solar Observatory, California TON and GONG+ sites"
caii,VSO,CAII,Kanzelhöhe Ca II k Instrument
cds,VSO,CDS,Coronal Diagnostic Spectrometer
celias,VSO,CELIAS,"Charge, Element, and Isotope Analysis System"
cerrotololo,VSO,Cerro Tololo,"Cerro Tololo, Chile GONG+ site"
chp,VSO,chp,Chromospheric Helium-I Imaging Photometer


sunpy also now provides tab completion to auto-fill the attribute name

In [13]:
a.Instrument.eit

<sunpy.net.attrs.Instrument(EIT: Extreme ultraviolet Imaging Telescope) object at 0x122268c70>

To search for certain wavelengths, we need to specify the input as an `astropy.Quantity` which is a the combination of a value and an associated unit. This is something is universal in the sunpy stack - that every physical input/output is a `Quantity`.

In [14]:
a.Wavelength(17.1*u.angstrom)

<sunpy.net.attrs.Wavelength(17.1, 17.1, 'Angstrom')>

## 3. Constructing a search query
 ### A simple query

Lets create a simple query to search for data from AIA over a particular time period

In [15]:
result = Fido.search(a.Time("2022-03-28 11:00", "2022-03-28 14:00"), 
                     a.Instrument("AIA"))

In [16]:
result

Start Time,End Time,Source,Instrument,Wavelength,Provider,Physobs,Wavetype,Extent Width,Extent Length,Extent Type,Size,Extra Flags,Info
,,,,Angstrom,,,,,,,Mibyte,,
Time,Time,str3,str3,float64[2],str4,str9,str6,str4,str4,str8,float64,str1,str106
2022-03-28 11:00:00.000,2022-03-28 11:09:13.000,SDO,AIA,335.0 .. 335.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,3038.47656,S,"AIA level 1, 4096x4096 [47 records] [0 eclipse] [0 darks] [2.901 to 2.901 exposure] [100.00 avg. percentd]"
2022-03-28 11:00:04.000,2022-03-28 11:09:17.000,SDO,AIA,193.0 .. 193.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,3038.47656,S,"AIA level 1, 4096x4096 [47 records] [0 eclipse] [0 darks] [2.000 to 2.000 exposure] [100.00 avg. percentd]"
2022-03-28 11:00:05.000,2022-03-28 11:00:06.000,SDO,AIA,4500.0 .. 4500.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,S,"AIA level 1, 4096x4096 [0.300 exposure] [100.00 percentd]"
2022-03-28 11:00:05.000,2022-03-28 11:09:18.000,SDO,AIA,304.0 .. 304.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,3038.47656,S,"AIA level 1, 4096x4096 [47 records] [0 eclipse] [0 darks] [2.902 to 2.902 exposure] [100.00 avg. percentd]"
2022-03-28 11:00:06.000,2022-03-28 11:09:19.000,SDO,AIA,131.0 .. 131.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,3038.47656,S,"AIA level 1, 4096x4096 [47 records] [0 eclipse] [0 darks] [2.901 to 2.901 exposure] [100.00 avg. percentd]"
2022-03-28 11:00:09.000,2022-03-28 11:09:22.000,SDO,AIA,171.0 .. 171.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,3038.47656,S,"AIA level 1, 4096x4096 [47 records] [0 eclipse] [0 darks] [2.000 to 2.000 exposure] [100.00 avg. percentd]"
2022-03-28 11:00:09.000,2022-03-28 11:09:10.000,SDO,AIA,211.0 .. 211.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,2973.82812,S,"AIA level 1, 4096x4096 [46 records] [0 eclipse] [0 darks] [2.901 to 2.901 exposure] [100.00 avg. percentd]"
2022-03-28 11:00:11.000,2022-03-28 11:09:12.000,SDO,AIA,94.0 .. 94.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,2973.82812,S,"AIA level 1, 4096x4096 [46 records] [0 eclipse] [0 darks] [2.901 to 2.901 exposure] [100.00 avg. percentd]"


Now lets make our query a bit more specific, say, say we only want one wavelength band from AIA. This can be achieved by specifying the `Wavelength` attribute within the search. The `a.Wavelength` attribute is passed as an `astropy.Quantity`:

In [17]:
result = Fido.search(a.Time("2022-03-28 11:00", "2022-03-28 14:00"), 
                     a.Instrument("AIA"), 
                     a.Wavelength(304*u.angstrom))

In [18]:
result

Start Time,End Time,Source,Instrument,Wavelength,Provider,Physobs,Wavetype,Extent Width,Extent Length,Extent Type,Size,Info
,,,,Angstrom,,,,,,,Mibyte,
Time,Time,str3,str3,float64[2],str4,str9,str6,str4,str4,str8,float64,str57
2022-03-28 11:00:05.000,2022-03-28 11:00:06.000,SDO,AIA,304.0 .. 304.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.902 exposure] [100.00 percentd]"
2022-03-28 11:00:17.000,2022-03-28 11:00:18.000,SDO,AIA,304.0 .. 304.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.902 exposure] [100.00 percentd]"
2022-03-28 11:00:29.000,2022-03-28 11:00:30.000,SDO,AIA,304.0 .. 304.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.902 exposure] [100.00 percentd]"
2022-03-28 11:00:41.000,2022-03-28 11:00:42.000,SDO,AIA,304.0 .. 304.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.902 exposure] [100.00 percentd]"
2022-03-28 11:00:53.000,2022-03-28 11:00:54.000,SDO,AIA,304.0 .. 304.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.902 exposure] [100.00 percentd]"
2022-03-28 11:01:05.000,2022-03-28 11:01:06.000,SDO,AIA,304.0 .. 304.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.902 exposure] [100.00 percentd]"
2022-03-28 11:01:17.000,2022-03-28 11:01:18.000,SDO,AIA,304.0 .. 304.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.902 exposure] [100.00 percentd]"
2022-03-28 11:01:29.000,2022-03-28 11:01:30.000,SDO,AIA,304.0 .. 304.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.902 exposure] [100.00 percentd]"


We can further specify this query by choosing the cadence (time-sampling) of the data we want to search for and download. This can be achieved by using the Sample attribute. Similar to the Wavelength attributes, this needs to be an astropy Quantity. Lets further specify the search above to only search for data with a cadence of 2 minutes.

In [19]:
result = Fido.search(a.Time("2022-03-28 11:00", "2022-03-28 14:00"), 
                     a.Instrument("AIA"), 
                     a.Wavelength(304*u.angstrom),
                     a.Sample(10*u.min))

In [20]:
len(result[0])

18

## 1.3 Downloading the data

Now we can show how data that is queried above can be downloaded. Once the data you have searched for (and filtered etc) is constructed into a query using `Fido.search`, you can then easily download them using `Fido.fetch`.

The data is downloaded via asynchronous and parallel download streams (via parfive), and also allows for failed data downloads to be recognized so that files can be re-requested if not downloaded.

Lets now look at how a `UnifiedResponse` from a `Fido.search` can be passed to `Fido.fetch` to download the data

In [ ]:
files = Fido.fetch(result)

Files Downloaded:   0%|          | 0/18 [00:00<?, ?file/s]

These files are downloaded to a local location set in the sunpy.config.file, which by default is ~/sunpy/data/. Fido.fetch returns a parfile.Results object which gives the path to where the files are downloaded to

In [ ]:
print(files[0])

You can also define what directory you want the files to be saved to by passing the directory path to the path keyword in Fido.fetch. For example, I want to download these files to a local directory `./AIA/<name_of_file>`

In [ ]:
Fido.fetch(result, path="./{instrument}/{file}")

## 1. 4 More complex queries

In addition to making a query to one client for one instrument, `Fido` allows the flexibility to search for data from multiple instruments, wavelengths, times etc, even when the data is being obtained through different clients.

This query can be constructed by using the pipe | operator, which joins queries together just like the OR operator.

Lets now make a query that searches for both GOES/XRS and AIA data over a particular time period

In [ ]:
result = Fido.search(a.Time("2022-03-28 11:00", "2022-03-28 14:00"), 
                     a.Instrument.xrs  | (a.Instrument.aia & a.Wavelength(304*u.angstrom)& a.Sample(10*u.min)))

In [ ]:
len(result)

In [ ]:
result

In [ ]:
result[0]

In [ ]:
result.all_colnames

Lets download the GOES XRS data first

In [ ]:
Fido.fetch(result[0, 0], path="./")

Now lets say we only want to download one AIA file at a particular time, we can also search the table for certain conditions. Lets say we just want the file that closest to 2022-03-28 12:00. 

In [ ]:
(np.abs(result[1]["Start Time"] - parse_time("2022-03-28 12:00"))).argmin()

In [ ]:
#Fido.fetch(result[1, 6], path="./")


## 1. 5 Using External Fido Clients 

Within `sunpy` core, we support a number of clients to common data providers. However, the `Fido` search interface is extensible such that external packages can write that their own clients that extend `Fido` in order to additional data sources. One such example is the `sunpy_soar` package which adds a client for the Solar Orbter Archive (SOAR).


## SOAR archive searching using sunpy!

In [ ]:
import sunpy_soar
from sunpy_soar.attrs import Product

Note that after importing `sunpy_soar`, the SOAR is now listed as a client that `Fido` will search.

In [ ]:
Fido

In [ ]:
eui_query = Fido.search(a.Time("2022-03-28 11:00", "2022-03-28 14:00"), 
                        a.soar.Product("EUI-FSI304-IMAGE"), 
                        a.Level(2))

In [ ]:
#eui_query

In [ ]:
Fido.fetch(eui_query, path="./{instrument}/{file}")

We can also search for other data products, for example the Solar Orbiter MAG

In [ ]:
mag_query = Fido.search(a.Time("2022-02-15", "2022-02-18"), 
                        a.soar.Product("MAG-RTN-NORMAL-1-MINUTE"), 
                        a.Level(2))

In [ ]:
mag_query

In [ ]:
ff = Fido.fetch(mag_query, path="./")